# Boggle Word Finder

In [30]:
import random
import numpy as np
import urllib.request

## Getting a Boggle Board

In [2]:
chars = 'abcdefghijklmnopqrstuvwxyz'

In [322]:
def get_random_char():
    return chars[random.randint(0,25)]

In [323]:
def get_random_boggle_board(x_dim=4, y_dim=4):
    return [[get_random_char() for i in range(x_dim)] for j in range(y_dim)]

In [327]:
boggle_board = get_random_boggle_board(); boggle_board

[['a', 'a', 's', 'u'],
 ['e', 'u', 'v', 'p'],
 ['z', 'q', 'b', 'o'],
 ['c', 'd', 'c', 'u']]

## Building a Dictionary Trie

The boggle board will use a trie as an index for quickly checking the words availble in the board

In [11]:
term_char = ''

Here is an example of what the trie looks like:

In [20]:
dict_trie = {'a': {term_char: True, 
                   'n': {term_char: True}, 
                   'b': {term_char: True}},
             }
dict_trie['a']['n'][term_char]

True

In [18]:
def build_dict_trie(dict_words):
    dict_trie = {}
    for word in dict_words:
        trie_node = dict_trie
        for char in word:
            if char not in trie_node:
                trie_node[char] = {}
            trie_node = trie_node[char]
        trie_node[term_char] = True
    return dict_trie            

In [285]:
url_for_words = 'https://raw.githubusercontent.com/first20hours/google-10000-english/master/google-10000-english-no-swears.txt'
# url_for_words = 'https://raw.githubusercontent.com/dwyl/english-words/master/words_alpha.txt'
en_words = [];
with urllib.request.urlopen(url_for_words) as response:
    en_words = response.read().decode('utf-8').split('\n')

In [286]:
en_words[:10]

['the', 'of', 'and', 'to', 'a', 'in', 'for', 'is', 'on', 'that']

In [287]:
en_words_longer_than_2_chars = [w for w in en_words if len(w) > 2]; en_words_longer_than_2_chars[:10]

['the', 'and', 'for', 'that', 'this', 'with', 'you', 'not', 'are', 'from']

In [288]:
dict_trie = build_dict_trie(en_words_longer_than_2_chars)

## Get all Words Available in a Boggle Board

In [272]:
def get_boggle_words(boggle_board):
    words = set()
    board_shape = np.shape(boggle_board)
    for j,i in np.ndindex(board_shape):
        next_char = boggle_board[j][i]
        words |= get_boggle_words_from_index(boggle_board, next_char, ((i,j),), dict_trie[next_char], board_shape)
    return words

In [273]:
def is_valid_new_index(new_idx, idxs, board_shape):
    i,j = new_idx
    y_dim, x_dim = board_shape
    return 0 <= i < x_dim and 0 <= j < y_dim and new_idx not in idxs

In [274]:
def get_boggle_words_from_index(boggle_board, prev_chars, idxs, trie_node, board_shape):
    i,j = idxs[-1]
    words = set()
    if term_char in trie_node:
        words.add(prev_chars)

    for x in [-1,0,1]:
        for y in [-1,0,1]:
            new_i = i + x
            new_j = j + y
            new_idx = (new_i, new_j);
            if is_valid_new_index(new_idx, idxs, board_shape):
                new_idxs = idxs + (new_idx,)
                next_char = boggle_board[new_j][new_i]
                if next_char in trie_node:
                    words |= get_boggle_words_from_index(boggle_board, 
                                                         prev_chars+next_char, 
                                                         new_idxs, 
                                                         trie_node[next_char], 
                                                         board_shape)
    return words

In [275]:
get_boggle_words([['a', 'z'],
                  ['b', 'n']])

{'abn', 'ban', 'nab', 'zan'}

In [290]:
get_boggle_words([['p', 'c'],
                  ['i', 'n'],
                  ['z', 'c']])

{'inc', 'pci', 'pic', 'pin', 'zinc', 'zip'}

In [278]:
'bottle' in en_words_longer_than_2_chars

True

In [263]:
boggle_board = get_random_boggle_board() 
boggle_board, get_boggle_words(boggle_board)

([['k', 'p', 'q', 'q'],
  ['z', 'h', 'v', 'x'],
  ['n', 'm', 'r', 's'],
  ['f', 't', 'j', 'o']],
 {'hrs', 'mhz', 'mrs', 'sort'})

## What About an Object Oriented Approach

In [150]:
class BoggleNode:
    def __init__(self, char=None):
        self.char = char or get_random_char();
        self.adjacent_nodes = [];
        
    def add_adjacent_node(self, node):
        self.adjacent_nodes.append(node)

In [249]:
class BoggleBoard:
    def __init__(self, shape=(4,4)):
        self.shape = shape
        self.board = [[BoggleNode() for i in range(shape[1])] for j in range(shape[0])]
        self._initialize_node_links_()
    
    def _initialize_node_links_(self):
        for j,i in np.ndindex(self.shape):
            for offset_j, offset_i in np.ndindex((3,3)):
                j2 = j - 1 + offset_j
                i2 = i - 1 + offset_i
                is_self_node = j2 == j and i2 == i
                if (not is_self_node) and 0 <= j2 < self.shape[0] and 0 <= i2 < self.shape[1]:
                    self.board[j][i].add_adjacent_node(self.board[j2][i2])
                    
    def find_all_words(self):
        words = set()
        for j,i in np.ndindex(self.shape):
            node = self.board[j][i]
            words |= self._find_words_from_node_(node, dict_trie[node.char], (node,));
        return words
        
    def _find_words_from_node_(self, node, trie_node, prev_nodes):
        words = set()
        if term_char in trie_node:
            words.add(''.join([node.char for node in prev_nodes]))
        for next_node in node.adjacent_nodes:
            next_char = next_node.char
            if next_char in trie_node:
                words |= self._find_words_from_node_(next_node, trie_node[next_char], prev_nodes + (next_node,))
        return words
    
    def __repr__(self):
        return '\n'.join([' '.join([self.board[j][i].char for i in range(self.shape[1])]) for j in range(self.shape[0])])

In [250]:
b = BoggleBoard(shape=(4,4))
b,b.find_all_words()

(p p d r
 i n g e
 c w z w
 d m e l,
 {'de',
  'der',
  'dpi',
  'drew',
  'ed',
  'edge',
  'el',
  'em',
  'er',
  'gdp',
  'greg',
  'grew',
  'i',
  'in',
  'inc',
  'ind',
  'ing',
  'me',
  'mel',
  'mem',
  'pgp',
  'pi',
  'pic',
  'picnic',
  'pin',
  'ping',
  're',
  'red',
  'reg',
  'we',
  'wed',
  'were',
  'win',
  'wind',
  'wing'})

## Making it easier to find words

In [299]:
char_counts = {}
for word in en_words_longer_than_2_chars:
    for char in word:
        char_counts[char] = 1 if char not in char_counts else char_counts[char] + 1
char_counts

{'a': 5299,
 'b': 1082,
 'c': 2945,
 'd': 2450,
 'e': 7524,
 'f': 884,
 'g': 1670,
 'h': 1381,
 'i': 5394,
 'j': 160,
 'k': 564,
 'l': 3171,
 'm': 1845,
 'n': 4752,
 'o': 4157,
 'p': 1961,
 'q': 115,
 'r': 4795,
 's': 4987,
 't': 4684,
 'u': 1887,
 'v': 820,
 'w': 599,
 'x': 237,
 'y': 1002,
 'z': 125}

In [310]:
num_to_char_dict = { i: chars[i] for i in range(len(chars)) }
num_to_char_dict

{0: 'a',
 1: 'b',
 2: 'c',
 3: 'd',
 4: 'e',
 5: 'f',
 6: 'g',
 7: 'h',
 8: 'i',
 9: 'j',
 10: 'k',
 11: 'l',
 12: 'm',
 13: 'n',
 14: 'o',
 15: 'p',
 16: 'q',
 17: 'r',
 18: 's',
 19: 't',
 20: 'u',
 21: 'v',
 22: 'w',
 23: 'x',
 24: 'y',
 25: 'z'}

In [312]:
running_total = 0
totals = []
for i in range(len(chars)):
    char_count = char_counts[num_to_char_dict[i]]
    running_total += char_count
    totals.append(running_total)
totals

[5299,
 6381,
 9326,
 11776,
 19300,
 20184,
 21854,
 23235,
 28629,
 28789,
 29353,
 32524,
 34369,
 39121,
 43278,
 45239,
 45354,
 50149,
 55136,
 59820,
 61707,
 62527,
 63126,
 63363,
 64365,
 64490]

In [313]:
import bisect

In [331]:
bisect.bisect_right([1,3,44,45,50], 44)

3

In [334]:
totals_max = max(totals)
def get_random_char_based_on_freq():
    rand = random.randint(0, totals_max - 1)
    freq_based_idx = bisect.bisect_right(totals, rand)
    return num_to_char_dict[freq_based_idx]

In [335]:
# Verify get_random_char_based_on_freq works properly
count_of_random_chars = { char: 0 for char in chars }
for i in range(totals_max):
    random_char = get_random_char_based_on_freq()
    count_of_random_chars[random_char] += 1
count_of_random_chars

{'a': 5460,
 'b': 1068,
 'c': 2958,
 'd': 2443,
 'e': 7523,
 'f': 871,
 'g': 1660,
 'h': 1377,
 'i': 5440,
 'j': 172,
 'k': 577,
 'l': 3240,
 'm': 1833,
 'n': 4701,
 'o': 4148,
 'p': 1978,
 'q': 118,
 'r': 4780,
 's': 5007,
 't': 4456,
 'u': 1869,
 'v': 850,
 'w': 606,
 'x': 233,
 'y': 1005,
 'z': 117}

In [336]:
get_random_char = get_random_char_based_on_freq

In [341]:
boggle_board = get_random_boggle_board(5,6) 
boggle_board, get_boggle_words(boggle_board)

([['n', 't', 'a', 'n', 'c'],
  ['i', 'b', 's', 'i', 'l'],
  ['f', 'k', 'n', 'b', 'r'],
  ['r', 'l', 'a', 't', 'm'],
  ['m', 'n', 'a', 't', 'a'],
  ['s', 'c', 'n', 'e', 'i']],
 {'abs',
  'acm',
  'acne',
  'air',
  'ala',
  'alan',
  'ana',
  'ann',
  'anna',
  'anne',
  'ant',
  'anti',
  'asin',
  'ask',
  'asn',
  'ata',
  'ate',
  'ati',
  'atm',
  'ban',
  'bank',
  'banks',
  'basic',
  'basin',
  'bat',
  'bias',
  'bin',
  'bit',
  'bits',
  'blink',
  'brian',
  'cal',
  'calm',
  'can',
  'canal',
  'cant',
  'cat',
  'cia',
  'cir',
  'cms',
  'cnet',
  'cnn',
  'eat',
  'ent',
  'fbi',
  'fin',
  'fit',
  'fits',
  'flat',
  'ian',
  'ibm',
  'inc',
  'incl',
  'ink',
  'ins',
  'int',
  'isa',
  'isbn',
  'ist',
  'its',
  'kate',
  'katie',
  'katrina',
  'kit',
  'kits',
  'lab',
  'labs',
  'lan',
  'lane',
  'lat',
  'late',
  'lbs',
  'lib',
  'libs',
  'link',
  'links',
  'lisa',
  'list',
  'mae',
  'mai',
  'mat',
  'mate',
  'matt',
  'mba',
  'mrna',
  'msn',
  '